In [142]:
# !pip install -U googlemaps

In [143]:
import googlemaps
from datetime import datetime, date
import pandas as pd

In [144]:
API_KEY = ""

In [145]:
gmaps = googlemaps.Client(key=API_KEY)

# origin = 'shettleston station glasgow'
# dest = 'Glasgow Central'
# today = datetime.now()
# dep_time = datetime(today.year, today.month, today.day, 9, 0, 0)

def get_gmaps_direction(origin, dest='Glasgow Central', dep_time=datetime(date.today().year, date.today().month, date.today().day, 9, 0, 0), mode='transit'):
    response = gmaps.directions(
        origin = origin,
        destination = dest,
        mode = mode,
        transit_mode = 'rail',
        region = 'uk',
        departure_time = dep_time,
        units = 'metric'
    )
    return response 

In [167]:
response = get_gmaps_direction("Barnhill Station, glasgow")

In [168]:
train_journey_time = 0
number_of_trains = 0
total_number_of_stops = 0
walking_time = 0
walking_distance = 0
origin_lng = 0
origin_lat = 0
total_time = 0
total_dist = 0

# response = get_gmaps_direction(row['station_1'])
    
legs = response[0]['legs'][0]

origin_lat = legs['start_location']['lat']
origin_lng = legs['start_location']['lng']

total_time = legs['duration']['value']
total_dist = legs['distance']['value']

for step in legs['steps']:
    if step['travel_mode'] == 'TRANSIT':
        number_of_trains += 1
        total_number_of_stops += step['transit_details']['num_stops']
        train_journey_time += step['duration']['value']
    elif step['travel_mode'] == 'WALKING':
        walking_distance += step['distance']['value']
        walking_time += step['duration']['value']

print("Origin Latitude:", origin_lat)
print("Origin Longitude:", origin_lng)
print("Total Travel Time (seconds):", total_time)
print("Total Distance (meters):", total_dist)
print("Train Journey Time (seconds):", train_journey_time)
print("Number of Trains:", number_of_trains)
print("Total Number of Stops:", total_number_of_stops)
print("Walking Time (seconds):", walking_time)
print("Walking Distance (meters):", walking_distance)

Origin Latitude: 55.877795
Origin Longitude: -4.2232438
Total Travel Time (seconds): 1366
Total Distance (meters): 5878
Train Journey Time (seconds): 660
Number of Trains: 1
Total Number of Stops: 5
Walking Time (seconds): 705
Walking Distance (meters): 861


In [149]:
df = pd.read_csv('output.csv')
train_journey_time_ = []
number_of_trains_ = []
total_number_of_stops_ = []
walking_time_ = []
walking_distance_ = []
origin_lat_ = []
origin_lng_ = []
total_time_ = []
total_dist_ = []

for i, row in df.iterrows():
    try:
        train_journey_time = 0
        number_of_trains = 0
        total_number_of_stops = 0
        walking_time = 0
        walking_distance = 0
    
        response = get_gmaps_direction(row['station_1'])
            
        legs = response[0]['legs'][0]
    
        origin_lat = legs['start_location']['lat']
        origin_lng = legs['start_location']['lng']
    
        total_time = legs['duration']['value']
        total_dist = legs['distance']['value']
    
        for step in legs['steps']:
            if step['travel_mode'] == 'TRANSIT':
                number_of_trains += 1
                total_number_of_stops += step['transit_details']['num_stops']
                train_journey_time += step['duration']['value']
            elif step['travel_mode'] == 'WALKING':
                walking_distance += step['distance']['value']
                walking_time += step['duration']['value']
    except Exception as e:
        print(e.message())
        train_journey_time = None
        number_of_trains = None
        total_number_of_stops = None
        walking_time = None
        walking_distance = None
        origin_lat = None
        origin_lng = None
        total_time = None
        total_dist = None
    
    train_journey_time_.append(train_journey_time)
    number_of_trains_.append(number_of_trains)
    total_number_of_stops_.append(total_number_of_stops)
    walking_time_.append(walking_time)
    walking_distance_.append(walking_distance)
    origin_lat_.append(origin_lat)
    origin_lng_.append(origin_lng)
    total_time_.append(total_time)
    total_dist_.append(total_dist)

df['train_journey_time'] = train_journey_time_
df['number_of_trains'] = number_of_trains_
df['total_number_of_stops'] = total_number_of_stops_
df['walking_time'] = walking_time_
df['walking_distance'] = walking_distance_
df['origin_lat'] = origin_lat_
df['origin_lng'] = origin_lng_
df['total_time'] = total_time_
df['total_dist'] = total_dist_

ApiError: NOT_FOUND

In [147]:
df.head(10)

,link,address,price,beds,bath,date_added,available_date,deposit,furnish_type,property_type,...,station_3_dist,train_journey_time,number_of_trains,total_number_of_stops,walking_time,walking_distance,origin_lat,origin_lng,total_time,total_dist
0,https://www.rightmove.co.uk/properties/1476730...,"Vista Park, Glasgow, G33 4QB","£1,475 pcm",3.0,NaN,24/04/2024,Now,Ask agent,Now,Terraced,...,1.4 miles,720,1,5,705,996,55.854485,-4.130828,1426,8850
1,https://www.rightmove.co.uk/properties/1494819...,"Yorkhill Street, Glasgow, G3","£1,200 pcm",2.0,1.0,today,26/07/2024,"£1,200",Furnished,Flat,...,0.7 miles,180,1,2,41,46,55.861402,-4.283080,221,1677
2,https://www.rightmove.co.uk/properties/1494802...,"Battlefield Gardens, Glasgow, G42",£950 pcm,1.0,1.0,today,22/07/2024,"£1,000",Furnished,Flat,...,0.6 miles,1140,1,7,150,148,55.826349,-4.261191,1290,8585
3,https://www.rightmove.co.uk/properties/1494765...,"Blantyre Street, Glasgow, G3",£925 pcm,1.0,1.0,yesterday,18/07/2024,"£1,000",Furnished,Flat,...,0.6 miles,660,1,6,378,457,55.870685,-4.300192,1039,4884
4,https://www.rightmove.co.uk/properties/1494765...,"Napiershall Street, Glasgow, G20",£925 pcm,2.0,1.0,yesterday,Now,£925,Unfurnished,Flat,...,0.6 miles,420,1,4,377,457,55.874336,-4.279577,798,3323


In [170]:
try:
    a = 10
    print(b)
except Exception as e:
    print("Error")

print(a)

Error
10


In [171]:
df

,link,address,price,beds,bath,date_added,available_date,deposit,furnish_type,property_type,station_1,station_1_dist,station_2,station_2_dist,station_3,station_3_dist
0,https://www.rightmove.co.uk/properties/1476730...,"Vista Park, Glasgow, G33 4QB","£1,475 pcm",3.0,NaN,24/04/2024,Now,Ask agent,Now,Terraced,Garrowhill Station,0.8 miles,Shettleston Station,0.8 miles,Carntyne Station,1.4 miles
1,https://www.rightmove.co.uk/properties/1494819...,"Yorkhill Street, Glasgow, G3","£1,200 pcm",2.0,1.0,today,26/07/2024,"£1,200",Furnished,Flat,Exhibition Centre Station,0.4 miles,Kelvinhall Station,0.5 miles,Hillhead Station,0.7 miles
2,https://www.rightmove.co.uk/properties/1494802...,"Battlefield Gardens, Glasgow, G42",£950 pcm,1.0,1.0,today,22/07/2024,"£1,000",Furnished,Flat,Mount Florida Station,0.2 miles,Langside Station,0.5 miles,Crosshill Station,0.6 miles
3,https://www.rightmove.co.uk/properties/1494765...,"Blantyre Street, Glasgow, G3",£925 pcm,1.0,1.0,yesterday,18/07/2024,"£1,000",Furnished,Flat,Kelvinhall Station,0.3 miles,Hillhead Station,0.5 miles,Exhibition Centre Station,0.6 miles
4,https://www.rightmove.co.uk/properties/1494765...,"Napiershall Street, Glasgow, G20",£925 pcm,2.0,1.0,yesterday,Now,£925,Unfurnished,Flat,Kelvinbridge Station,0.3 miles,St. George's Cross Station,0.3 miles,Charing Cross (Glasgow) Station,0.6 miles
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1039,https://www.rightmove.co.uk/properties/1485021...,"Kent Road, Glasgow, G3","£1,295 pcm",1.0,1.0,30/04/2024,Now,"£1,295",Furnished,Flat,Charing Cross (Glasgow) Station,0.1 miles,Anderston Station,0.3 miles,St. George's Cross Station,0.5 miles
1040,https://www.rightmove.co.uk/properties/1473995...,"Buccleuch Street, City Centre, Glasgow, G3",£900 pcm,1.0,1.0,29/04/2024,Now,£300,Part furnished,Flat,Charing Cross (Glasgow) Station,0.2 miles,St. George's Cross Station,0.3 miles,Cowcaddens Station,0.3 miles
1041,https://www.rightmove.co.uk/properties/1473892...,"Victoria Crescent Road, Glasgow, G12",£700 pcm,NaN,1.0,29/04/2024,Ask agent,"£1,050",Furnished,Flat,Hillhead Station,0.2 miles,Kelvinhall Station,0.4 miles,Partick Station,0.6 miles
1042,https://www.rightmove.co.uk/properties/1493997...,"Eastside, Kirkintilloch, East Dunbartonshire, G66",£700 pcm,2.0,1.0,29/04/2024,Now,£900,Furnished,Flat,Lenzie Station,1.4 miles,Stepps Station,3.6 miles,Robroyston Station,3.8 miles
